In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import minmax_scale

from pullDate_FullPeriod import pull_data , pull_seas 


# Python local connection to Oracle (herccrt) and Teradata (mosaic)
def connect_to_servers():
    from config import herccrt, mosaic, azure
    hcrt = herccrt().con()
    mos = mosaic().con()
    az = azure().con()
    return hcrt, mos, az

# jupyter notebook settings
import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20) # DON't Use None, it will show every row --> resulting in CRASH

hcrt, mos, az = connect_to_servers()

In [3]:
# choose directional market leg
orig = 'PHX'
dest = 'DFW'
### HAS UCLL:
# orig = 'DEN'
# dest = 'DFW'

new_market = False # change this to True if it is a new market

# choose fcst_id
fcst_id = 1


# constants
cabin = 'Y'
ulcc_list = ['NK','SY','F9'] # Spirit SunCountry Frontier 


# pull data time range
# pull_start = '2015-09-01'
# pull_end = '2023-12-30'

next_year_today = datetime.today() + timedelta(days=365)
yesterday =  datetime.today() - timedelta(days=1)

pull_start = '2017-09-01'
# pull_end = next_year_today.strftime("%Y-%m-%d")
pull_end = yesterday.strftime("%Y-%m-%d")

# Pre: pre-covid period, used for train and validation
Pre_start, Pre_end = '2017-09-01', '2020-01-30'
# Post: post-covid period, used for test
# Post_start, Post_end = '2021-07-01', '2022-06-30'
Post_start, Post_end = '2021-07-01', pull_end


# # !!! update here if use another fcst_id - NOT Sure what is this!
# fcst_start = 988
# fcst_end = 1619

In [4]:

def find_all_dest_given_leg(orig, hcrt):
    # Find cities given one leg:

    
    fcst_id_qry = f"""
    select Distinct LEG_DEST_S as dest
    from fcst.fcst_id_ref
    where LEG_ORIG_S = '{orig}'
    """
    fcst_id_df = pd.read_sql_query(fcst_id_qry, con=hcrt)

    return list(value[0] for value in fcst_id_df.values)


def get_fcst_given_leg(orig, dest, hcrt ):
    # Find fcst_id and start and end of the fcst time_bounds for a given orig dest

    fcst_id_qry = f"""
    select Distinct LEG_ORIG_S as orig, LEG_DEST_S as dest, FCST_ID as fcst_id,
            TIME_BAND_START as time_band_start, TIME_BAND_END as time_band_end
    from fcst.fcst_id_ref
    where 1=1 
    and LEG_ORIG_S = '{orig}'
    and LEG_DEST_S = '{dest}'
    order by 1,2,3,4
    """
    fcst_id_df = pd.read_sql_query(fcst_id_qry, con=hcrt)
   

    return fcst_id_df



In [5]:
def get_oag_data(orig,dest, pull_start, pull_end):

    oag_qry = f"""
    select DEP_AIRPRT_IATA_CD as orig, 
            ARVL_AIRPRT_IATA_CD as dest,
            LOCAL_DEP_DT as dep_date, 
            DEP_MINUTE_PAST_MDNGHT_QTY as dep_mam,
            FLIGHT_SCHD_PUBLSH_DT as snapshot_date,
            OPERAT_AIRLN_IATA_CD as airline, 
            OPERAT_FLIGHT_NBR as flt_id, -- Flight Number 
            EQUIP_COACH_CABIN_SEAT_QTY as seats, 
            ASMS_QTY as asm,
            EQUIP_COACH_CABIN_SEAT_QTY * MILE_GREAT_CIRCLE_DISTANC_QTY as asm_y -- ASM for Coach Cabin
    from PROD_INDSTR_FLIGHT_SCHD_VW.OAG_CURR  
    where 1=1
    and DEP_AIRPRT_IATA_CD = '{orig}'
    and ARVL_AIRPRT_IATA_CD = '{dest}' 
    and LOCAL_DEP_DT between '{pull_start}' and '{pull_end}'
    -- and LOCAL_DEP_DT = '2022-09-12'
    -- and OPERAT_AIRLN_IATA_CD = 'AA'
    and OPERAT_PAX_FLIGHT_IND = 'Y' -- new field that determines if record is a scheduled operating flight record
    and FLIGHT_OAG_PUBLSH_CD <> 'X' -- record is active and not cancelled 
    order by 1,2,3,4,5
    """

    oag_df = pd.read_sql(oag_qry, con=mos)


    # convert to datetime format
    oag_df['dep_date'] = pd.to_datetime(oag_df['dep_date'], format='%Y/%m/%d')

    # convert the dep_time before 3am to the previous dep_date
    # + Also you have ti change the dep_date for that one as well.
    oag_df['dep_mins'] = [val+24*60 if val<180 else val for val in oag_df['dep_mam']]
    oag_df['adj_dep_date'] = [date - dt.timedelta(days=1) if mam<180 else date for mam, date in zip(oag_df['dep_mam'],oag_df['dep_date'])]

    # add yr, mo, wk cols
    oag_df['yr'] = oag_df['adj_dep_date'].dt.year
    oag_df['mo'] = oag_df['adj_dep_date'].dt.month
    oag_df['wk'] = oag_df['adj_dep_date'].dt.isocalendar().week
    # add ulcc indicator
    oag_df['ulcc_ind'] = [1 if val in ulcc_list else 0 for val in oag_df['airline']]
    oag_df['seats_ulcc'] = [seats if val in ulcc_list else 0 for val , seats in zip(oag_df['airline'],oag_df['seats'])]


    return oag_df


def get_cap_data(orig,dest, pull_start, pull_end , cabin = 'Y'):
    cap_query = f"""
    select LEG_DEP_AIRPRT_IATA_CD as orig, 
            LEG_ARVL_AIRPRT_IATA_CD as dest,
            SCHD_LEG_DEP_DT as dep_date, 
            SCHD_LEG_DEP_TM as dep_time, 
            FILE_SNPSHT_DT as snapshot_date,
            LEG_CABIN_CD as cabin, 
            OPERAT_AIRLN_IATA_CD as airline, 
            MKT_FLIGHT_NBR as flt_id,  --Flight Number
            CABIN_CAPCTY_SEAT_QTY as seats, 
            CAB_ASM_QTY as asm, 
            CAB_TOT_RPM_QTY as rpm,
            CAB_TOT_REVNUE_AMT as rev, 
            CAB_TOT_PAX_QTY as pax
    from PROD_RM_BUSINES_VW.LIFE_OF_FLIGHT_LEG_CABIN 
    where 1=1
    and FILE_SNPSHT_DT = SCHD_LEG_DEP_DT-1 -- only extract the data one day before departure
    and LEG_DEP_AIRPRT_IATA_CD = '{orig}'
    and LEG_ARVL_AIRPRT_IATA_CD = '{dest}'
    and SCHD_LEG_DEP_DT between '{pull_start}' and '{pull_end}'
    -- and SCHD_LEG_DEP_DT = '2022-09-12' 
    and LEG_CABIN_CD = '{cabin}'
    order by 1,2,3,4,5
    """

    cap_df = pd.read_sql(cap_query, con=mos)


    # convert to datetime format
    cap_df['dep_date'] = pd.to_datetime(cap_df['dep_date'], format='%Y/%m/%d')
    # cap_df['dep_time'] = pd.to_datetime(cap_df['dep_date']+cap_df['dep_time'], format='%H:%M:%S')
    cap_df['dep_time'] = pd.to_datetime(cap_df['dep_date'].astype(str)+' '+cap_df['dep_time'].astype(str), format='%Y/%m/%d %H:%M:%S')

    # count the minutes from mid-night
    cap_df['dep_mins'] = pd.DatetimeIndex(cap_df['dep_time']).hour*60 + pd.DatetimeIndex(cap_df['dep_time']).minute
    # convert the dep_time before 3am to the previous dep_date
    # + Also you have to adjust the dep_date for that one as well.
    cap_df['adj_dep_date'] = [date - dt.timedelta(days=1) if mam<180 else date for mam, date in zip(cap_df['dep_mins'],cap_df['dep_date'])]
    cap_df['dep_mins'] = [val+24*60 if val<180 else val for val in cap_df['dep_mins']]


    # add yr, mo, wk
    cap_df['yr'] = cap_df['adj_dep_date'].dt.year
    cap_df['mo'] = cap_df['adj_dep_date'].dt.month
    cap_df['wk'] = cap_df['adj_dep_date'].dt.isocalendar().week

    return cap_df

In [6]:
def oag_per_day(oag_df):
    # get OA Cap per day.

    # groupby for the entire market (So we can calculate the Shares) - Sina changed it to per day.
    gp_cols = ['adj_dep_date','airline'] 
    agg_cols = {'seats':'sum', 'asm_y':'sum','flt_id':'count','ulcc_ind':'sum', 'seats_ulcc': 'sum'}

    oag_kl_Per_airline_Day = oag_df.groupby(gp_cols).agg(agg_cols).reset_index()

    # change flt_id name as flt_ct, and asm_y to asm (since we are focusing on y cabin)
    oag_kl_Per_airline_Day.rename(columns={'flt_id':'flt_ct','ulcc_ind':'ulcc_ind_mkt', 'asm_y':'asm' }, inplace=True) 

    # Now lets aggrigate all airlines to have infomration on all airlines.
    gp_cols = ['adj_dep_date'] 
    agg_cols = {'seats':'sum', 'asm':'sum','flt_ct':'sum','ulcc_ind_mkt':'sum', 'seats_ulcc': 'sum'}

    oag_kl_total_Per_Day = oag_kl_Per_airline_Day.groupby(gp_cols).agg(agg_cols).reset_index()

    # Filter American flights into a seperate view

    filter0 = oag_kl_Per_airline_Day['airline'] == 'AA'
    oag_kl_Per_american_Day = oag_kl_Per_airline_Day[filter0]

    # Drop unrelated infomration
    oag_kl_Per_american_Day.drop(columns=['ulcc_ind_mkt','seats_ulcc','airline'],inplace=True)

    # Merge the AA data with the aggrigate data 
    oag_kl_total_Per_Day_and_AA = pd.merge(oag_kl_total_Per_Day,oag_kl_Per_american_Day,on = gp_cols, how='left',suffixes=('_All', '_AA'))
    oag_kl_total_Per_Day_and_AA.rename(columns={'ulcc_ind_mkt':'flt_ct_ulcc'}, inplace=True) 

    # Calculate OA data.
    oag_kl_total_Per_Day_and_AA['seats_OA'] = oag_kl_total_Per_Day_and_AA['seats_All'] - oag_kl_total_Per_Day_and_AA['seats_AA'] -  oag_kl_total_Per_Day_and_AA['seats_ulcc'] 
    oag_kl_total_Per_Day_and_AA['flt_ct_OA'] = oag_kl_total_Per_Day_and_AA['flt_ct_All'] - oag_kl_total_Per_Day_and_AA['flt_ct_AA'] - oag_kl_total_Per_Day_and_AA['flt_ct_ulcc']

    # Reformat the data
    oag_kl_total_Per_Day_and_AA = oag_kl_total_Per_Day_and_AA.loc[:,['adj_dep_date','seats_AA','seats_OA','seats_ulcc','seats_All','flt_ct_AA' , 'flt_ct_OA' ,'flt_ct_ulcc' , 'flt_ct_All' ,'asm_AA','asm_All']]

    return oag_kl_total_Per_Day_and_AA


def oag_per_fcst(oag_df, fcst_start, fcst_end ):
    # OAG only keep the dep_mins in the fcst_id
    oag_df2 = oag_df[(oag_df['dep_mins']>=fcst_start) & (oag_df['dep_mins']<=fcst_end)]


    # here we only care about AA.
    gp_cols = ['adj_dep_date','airline']
    agg_cols = {'seats':'sum', 'asm_y':'sum','flt_id':'count','ulcc_ind':'sum', 'seats_ulcc': 'sum' }

    oag_kl = oag_df2.groupby(gp_cols).agg(agg_cols).reset_index()
    # change flt_id name as flt_ct
    oag_kl.rename(columns={'flt_id':'flt_ct', 'asm_y':'asm'}, inplace=True)
    oag_kl
    # groupby for the entire market (So we can calculate the Shares) 
    gp_cols = ['adj_dep_date'] 
    agg_cols = {'seats':'sum','asm_y':'sum' ,'flt_id':'count','ulcc_ind':'sum', 'seats_ulcc': 'sum'}

    oag_kl_AAOA = oag_df2.groupby(gp_cols).agg(agg_cols).reset_index()
    # change flt_id name as flt_ct
    oag_kl_AAOA.rename(columns={'flt_id':'flt_ct' , 'ulcc_ind':'ulcc_count', 'asm_y':'asm' }, inplace=True) 


    # Filter the oag_df2 to show theAA Cap (per fcst span) and merge AAOA Cap (on the specific fcst).
    filterAA = oag_kl['airline'] == 'AA'
    oag_kl = oag_kl[filterAA]
    oag_kl.drop(columns=['ulcc_ind','seats_ulcc','airline'],inplace=True)


    oag_kl = pd.merge(oag_kl,oag_kl_AAOA,on = gp_cols, how='left',suffixes=('_AA_fcst', '_All_fcst'))


    # add OA Cap
    oag_kl['seats_OA_fcst'] = oag_kl['seats_All_fcst']-oag_kl['seats_AA_fcst']-oag_kl['seats_ulcc']
    oag_kl['asm_OA_fcst'] = oag_kl['asm_All_fcst']-oag_kl['asm_AA_fcst']
    oag_kl['flt_ct_OA_fcst'] = oag_kl['flt_ct_All_fcst']-oag_kl['flt_ct_AA_fcst']-oag_kl['ulcc_count']

    # # add AA market share
    # oag_kl['seats_share'] = oag_kl['seats_AA_fcst']/oag_kl['seats_AAOA']
    # oag_kl['asm_share'] = oag_kl['asm_AA_fcst']/oag_kl['asm_AAOA']
    # oag_kl['flt_ct_share'] = oag_kl['flt_ct_AA_fcst']/oag_kl['flt_ct_AAOA']
    # # add seats_per_flt
    # oag_kl['seats_per_flt_AA'] = oag_kl['seats_AA']/oag_kl['flt_ct_AA']
    # oag_kl['seats_per_flt_OA'] = oag_kl['seats_OA']/oag_kl['flt_ct_OA']
    # oag_kl['seats_per_flt_AAOA'] = oag_kl['seats_AAOA']/oag_kl['flt_ct_AAOA']

    oag_kl['fcst_start'] = fcst_start
    oag_kl['fcst_end'] = fcst_end
    oag_kl.rename(columns={'seats_ulcc':'seats_ulcc_fcst' , 'ulcc_count':'flt_ct_ulcc_fcst' }, inplace=True) 

    oag_kl = oag_kl.loc[:,['adj_dep_date', 'fcst_start', 'fcst_end','seats_AA_fcst','seats_OA_fcst','seats_ulcc_fcst','seats_All_fcst','flt_ct_AA_fcst' , 'flt_ct_OA_fcst' ,'flt_ct_ulcc_fcst' , 'flt_ct_All_fcst' ,'asm_AA_fcst','asm_All_fcst']]

    return oag_kl

def normalize_oag_kl_fcst_total(oag_kl_fcst_total): 
    
    # Drop Null
    oag_kl_fcst_total.dropna(inplace=True)

    # # remove the 
    # oag_kl_fcst_total.drop(columns=['seats','asm','flt_ct' , 'fcst_start' , 'fcst_end'],inplace=True)

    # Normalize Cap features
    norm_cols = [
        'seats_AA_fcst',
        'seats_OA_fcst', 'seats_ulcc_fcst', 'seats_All_fcst', 'flt_ct_AA_fcst',
        'flt_ct_OA_fcst', 'flt_ct_ulcc_fcst', 'flt_ct_All_fcst', 'asm_AA_fcst',
        'asm_All_fcst', 'seats_AA', 'seats_OA', 'seats_ulcc', 'seats_All',
        'flt_ct_AA', 'flt_ct_OA', 'flt_ct_ulcc', 'flt_ct_All', 'asm_AA',
        'asm_All']

    oag_kl_fcst_total[norm_cols] = minmax_scale(oag_kl_fcst_total[norm_cols])
    
    return oag_kl_fcst_total


def aa_cap_fcst(cap_df,fcst_start, fcst_end ):
    # AA Cap only keep the dep_mins in the fcst_id
    cap_df2 = cap_df[(cap_df['dep_mins']>=fcst_start) & (cap_df['dep_mins']<=fcst_end)]

    # get AA Cap
    gp_cols = ['dep_date']   
    agg_cols = {'seats':'sum','asm':'sum','flt_id':'count','rpm':'sum','rev':'sum','pax':'sum'}

    cap_kl = cap_df2.groupby(gp_cols).agg(agg_cols).reset_index()
    cap_kl.rename(columns={'flt_id':'flt_ct'}, inplace=True)

    # add other Cap features
    cap_kl['rasm'] = cap_kl['rev']/cap_kl['asm']
    cap_kl['yield'] = cap_kl['rev']/cap_kl['rpm']
    cap_kl['load_fac'] = cap_kl['rpm']/cap_kl['asm']

    # replace N/A with 0
    # print(cap_kl.isnull().sum())
    cap_kl = cap_kl.replace(np.nan, 0)
    # print(cap_kl.isnull().sum())

    return cap_kl



def merge_oag_aacap(oag_kl,cap_kl): 
    # merge OAG and AA Cap data

    oag_kl.rename(columns={'adj_dep_date':'dep_date'}, inplace=True)

    oag_cap_kl = pd.merge(cap_kl,oag_kl,on=['dep_date'],how='left',suffixes=('_cap', '_oag'))

    # print(oag_cap_kl.isnull().sum())
    oag_cap_kl.dropna(inplace=True)

    # remove the 
    oag_cap_kl.drop(columns=['seats','asm','flt_ct' , 'fcst_start' , 'fcst_end'],inplace=True)

    # Normalize Cap features
    norm_cols = ['rpm', 'rev', 'pax', 'rasm',
           'yield', 'load_fac', 'seats_AA_fcst',
           'seats_OA_fcst', 'seats_ulcc_fcst', 'seats_All_fcst', 'flt_ct_AA_fcst',
           'flt_ct_OA_fcst', 'flt_ct_ulcc_fcst', 'flt_ct_All_fcst', 'asm_AA_fcst',
           'asm_All_fcst']

    oag_cap_kl[norm_cols] = minmax_scale(oag_cap_kl[norm_cols])
    
    return oag_cap_kl





In [7]:
def create_group_id(df):
    """
    Identify and padding the unique group
    Each group contains 14 rows (local/flow, 7 fcst_perd) x 10 cols (frac_closure).
    Each flight is a group!
    """


    df = df.sort_values(['snapshotDate','origin','destination',
                                'forecastId','flightDepartureDate','forecastDayOfWeek','poolCode',
                                 'cabinCode','localFlowIndicator'])

    # 'GroupBy.cumcount': Number each item in each group from 0 to (the length of that group - 1).
    # '== 0' returns True or False
    # '.astype(int)' ,convert True/False to 1/0
    df['groupID'] = (df.groupby(['snapshotDate','origin','destination',
                                'forecastId','flightDepartureDate','forecastDayOfWeek','poolCode',
                                 'cabinCode']).cumcount() == 0).astype(int)
    # assign each unique group a new group id
    # groupID==1 will be a new id (+1), groupID==0 will indicate the same id for the same group
    df['groupID'] = df['groupID'].cumsum()

    # Full Hisxtory Pre Fixing
    # count the num of 'forecastPeriod' in each group
    # because some group might not have all 7 'forecastPeriod'
    df['fullHistory'] = df.groupby(['groupID'])["forecastPeriod"].transform("count")
    # Indicator it is part of history and not a pad
    df['real'] = 1
    return df


def empty_group():
    # Creating an empty group to be used for padding.
    fullKeysArray = np.zeros( (14, 50) ) # 'fullKeys' uses 50 columns 

    # frac_closure = 0: fully closed with 0 traffic
    for i in range(0,fullKeysArray.shape[0]): 
        if i <= 6: # Flow: 0 ~ 6
            fullKeysArray[i][0] = 1 # 'localFlowIndicator' column
            fullKeysArray[i][1] = i + 1 # 'forecastPeriod' column: 1 ~ 7
            fullKeysArray[i][2:12] = 1 # 'fracClosure' column all set to 1 
        else: # Local: 7 ~ 13s
            fullKeysArray[i][0] = 0 # 'localFlowIndicator' column
            fullKeysArray[i][1] = i - 6 # 'forecastPeriod' column: 1 ~ 7
            fullKeysArray[i][2:12] = 1 # 'fracClosure' column: all set to 1 

    fullKeys = pd.DataFrame(fullKeysArray)        
    fullKeys.columns = ['localFlowIndicator','forecastPeriod'] + [f"fracClosure_{i}" for i in range(1,11)] + [f"trafficActual_{i}" for i in range(1,11)] + [f"trafficActualAadv_{i}" for i in range(1,11)] + ['holiday','H1','H2','H3','HL','weekNumber','week_x','week_y','dow_x','dow_y','avgtraffic','avgtrafficopenness','avgrasm','dowavgtraffic','dowavgtrafficopenness','dowavgrasm','groupID','fullHistory']
    fullKeys['localFlowIndicator'] = ['F' if lfi == 1 else 'L' for lfi in fullKeys['localFlowIndicator']]
    return fullKeys

def padding_groups(df):
    fullKeys = empty_group()
    
    groupbyColumns = ['snapshotDate','origin','destination','forecastId','flightDepartureDate',
                  'forecastDayOfWeek','poolCode','cabinCode']
    grouped = df.groupby(groupbyColumns)
    
    merged_list = []
    count_rows_misskey = 0
    for g in grouped:

        # g[0] is the directory key and g[1] is the value (actual data)
        # identify the cells of g[1] that not in fullKeys
        # g[1][~g[1].isin(fullKeys)] 

        # identify the missing keys in g[1]
        key = g[1][['localFlowIndicator','forecastPeriod']]
        missingKeys = fullKeys[~fullKeys[['localFlowIndicator','forecastPeriod']].apply(tuple,1).isin(key.apply(tuple,1))]
        count_rows_misskey += missingKeys.shape[0]

        # append the missing keys under the data
        fullHistory = pd.concat([g[1],missingKeys])

        # use 0 to indicate padding data
        fullHistory['real'].fillna(0, inplace=True) 

        # fill the data with missing keys
        fullHistory = fullHistory.fillna(method='ffill')
        merged_list.append(fullHistory)

    # merge all data across 'flightDepartureDate'   
    out = pd.concat(merged_list)
    out = out.sort_values(['snapshotDate','origin','destination',
                                'forecastId','flightDepartureDate','forecastDayOfWeek','poolCode',
                                 'cabinCode','localFlowIndicator'])
    
    
    # Get full history and then concat fake history (padding) from above 
    post = out.copy()
    post = post.sort_values(['snapshotDate','origin','destination',
                                'forecastId','flightDepartureDate','forecastDayOfWeek','poolCode',
                                 'cabinCode','localFlowIndicator'])


    post['groupID'] = (post.groupby(['snapshotDate','origin','destination',
                                'forecastId','flightDepartureDate','forecastDayOfWeek','poolCode',
                                 'cabinCode']).cumcount()==0).astype(int)
    post['groupID'] = post['groupID'].cumsum()
    # Full Hisotyr Pre Fixing
    post['fullHistory'] = post.groupby(['groupID'])["forecastPeriod"].transform("count")

    post['flightDepartureDate'] = pd.to_datetime(post['flightDepartureDate'], format='%Y/%m/%d')

    
    return post

def group_and_pad(df):
    return padding_groups(create_group_id(df))
    
    
    

In [8]:
def get_tensors(DataFarame, sea_col_Cap, test = False, time_series = True , use_channels = False , seasenality_one_dimension = True ,  window = 10):
    
    len_sea_cap = len(sea_col_Cap)
            
    
    # fractional closure
    PRE_FC_L = DataFarame[['fracClosure_' + str(i + 1) for i in range(10)]].values.astype('float32')
    # seasonality
    PRE_Sea_L = DataFarame[sea_col_Cap].values.astype('float32')
    # actual traffic
    if test:
        Traffic = False
    else:
        PRE_Traf_L = DataFarame[['trafficActual_' + str(i + 1) for i in range(10)]].values.astype('float32')
    
    
    # reshape the data for CNNLSTM model
    FC = PRE_FC_L.reshape(int(PRE_FC_L.shape[0]/14), 1, 14, 10)
    Seasenality = PRE_Sea_L.reshape(int(PRE_Sea_L.shape[0]/14), 1,14, len_sea_cap)
    if not test:
        Traffic = PRE_Traf_L.reshape(int(PRE_Traf_L.shape[0]/14), 1, 14, 10)
    
    # Remove Duplicates (from 2d to 1d vector)
    if seasenality_one_dimension:
        Seasenality =np.delete(Seasenality, slice(13), 2).reshape(Seasenality.shape[0],len_sea_cap)
    
    
    if use_channels:
        FC = FC.reshape(len(FC),2,7,10)
        if not test:
            Traffic = Traffic.reshape(len(Traffic),2,7,10)
    
    # Change FC shape to refelect time series:
    # print(FC.shape)
    if time_series:
        time_series_widow= list()
        Seasenality_times = list()
        for i in range(window,len(FC)):
            # print(FC[i-window:i].shape)
            time_series_widow.append(FC[i-window:i].reshape(window,2,7,10))
            # print((Seasenality[i-window:i].shape))
            Seasenality_times.append(Seasenality[i-window:i])
        FC = np.array(time_series_widow)
        Seasenality = np.array(Seasenality_times)
        
        # Since the 1st window size data points are removed:
        # Seasenality = Seasenality[window:]
        if not test:
            Traffic = Traffic[window:]
        
        
    return FC, Seasenality, Traffic
   
    
    
def get_train_test_samples(Data_PRE, Data_POST, Data_FUTURE, sea_col_Cap , train_val_percentage = 0.9 , time_series = False, use_channels = False,  seasenality_one_dimension = True , window = 0  ):
    
    PRE_FC , PRE_Seas , PRE_Traf  = get_tensors(Data_PRE, sea_col_Cap , test = False, time_series =time_series , use_channels = use_channels , seasenality_one_dimension=seasenality_one_dimension , window =window)
    POST_FC , POST_Seas , _  = get_tensors(Data_POST, sea_col_Cap, test = True, time_series = time_series, use_channels = use_channels, seasenality_one_dimension=seasenality_one_dimension , window = window )
    # TODO: FOR NOW WE ARE NOT USING THE FUTURE:
    FUTURE_FC , FUTURE_Seas , _  = get_tensors(Data_FUTURE, sea_col_Cap ,test = True, time_series = time_series, use_channels = use_channels, seasenality_one_dimension=seasenality_one_dimension , window = window)
    
    # 80, 20 --> 
    # TODO: THIS SHOULD BE CHANGED TO RANDOMIZED.
    train_val_cutoff = round(PRE_FC.shape[0]*train_val_percentage) 


    # prepare train/val/test datasets
    PRE_FC_train = PRE_FC[:train_val_cutoff, :]
    PRE_FC_val = PRE_FC[train_val_cutoff:, :]

    PRE_Seas_train = PRE_Seas[:train_val_cutoff, :]
    PRE_Seas_val = PRE_Seas[train_val_cutoff:, :]

    PRE_Traf_train = PRE_Traf[:train_val_cutoff, :]
    PRE_Traf_val = PRE_Traf[train_val_cutoff:, :]



    train = [PRE_FC_train,PRE_Seas_train,PRE_Traf_train]
    val = [PRE_FC_val,PRE_Seas_val,PRE_Traf_val]
    test = [POST_FC,POST_Seas]
    
    return train, val, test
    

In [9]:
def randPeriod(prdMaps):
    # print(x)
    random_period = np.random.randint(1,7)
    rrd_start, rrt_end = prdMaps[prdMaps['FORECASTPERIOD']==random_period].loc[:,["RRD_START", "RRD_END"]].values[0]
    random_day = np.random.randint(rrd_start, rrt_end )
    return random_period , random_day


def tf_timeseries_masking( tf_tensors , data_index , prdMaps , window  ):
    """This function will generate masked time-seried terrafic data."""
    
    random_period , random_day_to_dept = randPeriod(prdMaps)
    # print(random_period , random_day_to_dept )
    arr =  prdMaps.iloc[:,3].values
    
    # output = tf_tensors[data_index].copy()
    test_tensors = tf_tensors.copy()
    test_tensors[data_index][:,:random_period,] = -1
    
    
    max_bond_period = random_day_to_dept
    min_bond_period = arr[random_period-1]
    remaining_window = window - 1
    current_index = data_index
    max_min_range = max_bond_period - min_bond_period
    current_period = random_period

    if  max_min_range < remaining_window:
        while max_min_range  <= remaining_window:
            # print(current_index-max_min_range,current_index)
            test_tensors[current_index-max_min_range:current_index,:,:current_period,] = -1
            current_period -= 1
            if current_period == 0:
                break
            current_index -= max_min_range
            remaining_window -=  max_min_range
            max_bond_period -= max_min_range
            min_bond_period = arr[current_period-1]
            max_min_range = max_bond_period - min_bond_period
            # reaching Today date:

    if max_min_range >= remaining_window:
        # print(current_index-max_min_range,current_index)
        test_tensors[current_index-remaining_window:current_index,:,:current_period,] = -1
    
    return test_tensors[data_index+1-window:data_index+1] 


def get_tensors2(DataFarame, sea_col_Cap, prdMaps= None , FC_time_series = True , traffic_time_series = False ,  use_channels = False , seasenality_one_dimension = True ,  window = 10, DOW=False):
    
    len_sea_cap = len(sea_col_Cap)
            
    
    # fractional closure
    PRE_FC_L = DataFarame[['fracClosure_' + str(i + 1) for i in range(10)]].values.astype('float32')
    # seasonality
    PRE_Sea_L = DataFarame[sea_col_Cap].values.astype('float32')
    # actual traffic
    PRE_Traf_L = DataFarame[['trafficActual_' + str(i + 1) for i in range(10)]].values.astype('float32')
    
    
    # reshape the data for CNNLSTM model
    FC = PRE_FC_L.reshape(int(PRE_FC_L.shape[0]/14), 1, 14, 10)
    Seasenality = PRE_Sea_L.reshape(int(PRE_Sea_L.shape[0]/14), 1,14, len_sea_cap)
    Traffic = PRE_Traf_L.reshape(int(PRE_Traf_L.shape[0]/14), 1, 14, 10)
    
    # Remove Duplicates (from 2d to 1d vector)
    if seasenality_one_dimension:
        Seasenality =np.delete(Seasenality, slice(13), 2).reshape(Seasenality.shape[0],len_sea_cap)
    
    
    if use_channels:
        FC = FC.reshape(len(FC),2,7,10)
        Traffic = Traffic.reshape(len(Traffic),2,7,10)
    
    # Change FC shape to refelect time series:
    # print(FC.shape)
    if FC_time_series:
        time_series_widow= list()
        Seasenality_times = list()
        for i in range(window,len(FC)):
            # print(FC[i-window:i].shape)
            time_series_widow.append(FC[i-window:i].reshape(window,2,7,10))
            # print((Seasenality[i-window:i].shape))
            Seasenality_times.append(Seasenality[i-window:i])
        FC = np.array(time_series_widow)
        Seasenality = np.array(Seasenality_times)
        
        # Since the 1st window size data points are removed:
        # Seasenality = Seasenality[window:]
        Traffic = Traffic[window:]
    
    elif traffic_time_series:
        traffic_time_series_window = list()
        Seasenality_times = list()
        for i in range(window,len(Traffic)):
            # Find Random period and random day:
            if DOW:
                tf_window_masked = tf_timeseries_masking_DOW( Traffic  , i , prdMaps , window  )
            else:
                tf_window_masked = tf_timeseries_masking( Traffic  , i , prdMaps , window  )
            traffic_time_series_window.append(tf_window_masked)
            # Seasenality_times.append(Seasenality[i-window:i])
        TF_time = np.array(traffic_time_series_window)
        # Seasenality = np.array(Seasenality_times)
        Seasenality = Seasenality[window:]
        FC = FC[window:]
        
        Traffic = Traffic[window:]
            
        return FC, Seasenality, Traffic, TF_time
    
        
    return FC, Seasenality, Traffic, _




def floorSearch(arr, low, high, x):
 
    # If low and high cross each other
    if (low > high):
        return -1
 
    # If last element is smaller than x
    if (x >= arr[high]):
        return high
 
    # Find the middle point
    mid = int((low + high) / 2)
 
    # If middle point is floor.
    if (arr[mid] == x):
        return mid
 
    # If x lies between mid-1 and mid
    if (mid > 0 and arr[mid-1] <= x
            and x < arr[mid]):
        return mid - 1
 
    # If x is smaller than mid,
    # floor must be in left half.
    if (x < arr[mid]):
        return floorSearch(arr, low, mid-1, x)
 
    # If mid-1 is not floor and x is greater than
    # arr[mid],
    return floorSearch(arr, mid + 1, high, x)


def tf_timeseries_masking_DOW( tf_tensors , data_index , prdMaps , window  ):
    """This function will generate masked time-seried terrafic data, and is based on DOW."""
    
    random_period , random_day_to_dept = randPeriod(prdMaps)
    arr =  prdMaps.iloc[:,3].values
    test_tensors = tf_tensors.copy()
    
    day_to_dept = random_day_to_dept
    current_index = data_index
    
    for i in range(0,window):
        # Move back 7 days in each iter.
        day_to_dept = random_day_to_dept - i*7 
        # Get the period of that day to dept.
        flrs = floorSearch(arr, 0, 6, day_to_dept)
        current_period = flrs+1
        # If we get today, will break the loop. and use all the values (no masking)
        if current_period == 0:
            break
        # mask the values
        test_tensors[current_index,:,:current_period,] = -1
        
        # Update index:
        current_index -= 1
        
    return test_tensors[data_index+1-window:data_index+1]    
    

        


# def dow_get_tensors2(DataFarame , sea_col_Cap, prdMaps= None  ,  test = False, time_series = True,  use_channels = False , window = 10):   
def dow_get_tensors2(DataFarame , sea_col_Cap, prdMaps= None  ,  FC_time_series = True , traffic_time_series = False ,  use_channels = False , seasenality_one_dimension = True ,  window = 10):
    DOW = True
    FC_dow , Seasenality_dow, Traffic_dow ,  TF_time_dow  = list(), list(), list(), list()
    
    for i in DataFarame.loc[ :,	['forecastDayOfWeek' ]].drop_duplicates().values:
        # filter_y = DataFarame['dow_y' ] == i[1] 
        # filter_x = DataFarame['dow_x'] == i[0] 
        filter_dow =  DataFarame['forecastDayOfWeek'] == i[0] 
        # print(filter_y.shape , filter_x.shape)
        # print(i)
        Data_dow =DataFarame[filter_dow]
        # print(Data_dow.shape)
        FC, Seasenality, Traffic, TF_time= get_tensors2(Data_dow, sea_col_Cap, prdMaps  , FC_time_series  , traffic_time_series ,  use_channels  , seasenality_one_dimension  ,  window, DOW )
        FC_dow.append(FC)
        Seasenality_dow.append(Seasenality)
        Traffic_dow.append(Traffic)
        if traffic_time_series:
            TF_time_dow.append(TF_time)
    
    # Then Concat together, now each datapoint is based on DOW.
    FC_dow = [ i  for i in FC_dow if i.shape!=(0,)]
    Seasenality_dow = [ i  for i in Seasenality_dow if i.shape!=(0,)]
    # Traffic_dow = [ i  for i in Traffic_dow if i.shape!=(0,)]
    
    if traffic_time_series:
        TF_time_dow = [ i  for i in TF_time_dow if i.shape!=(0,)]
        TF_time_dow = np.concatenate(TF_time_dow)
    
    FC_dow = np.concatenate(FC_dow)
    Seasenality_dow = np.concatenate(Seasenality_dow)
    Traffic_dow = np.concatenate(Traffic_dow)
    
    return FC_dow , Seasenality_dow, Traffic_dow , TF_time_dow




def get_prdMaps(orig, dest, hcrt):
    prdMaps = pd.read_sql(f"""select DISTINCT leg_orig as origin, leg_dest as destination, fcst_period as forecastPeriod, rrd_band_start_i as rrd_start, rrd_band_end_i as rrd_end
                            from market_xref a 
                            join FCST.FCST_PERIOD_REF b 
                            on a.infl_period_id = b.FCST_PERIOD_ID
                            where 1=1 
                            and cabin_code = 'Y'
                            and leg_orig = '{orig}'
                            and leg_dest = '{dest}'
                            ORDER BY forecastPeriod
                            """, con = hcrt)
    return prdMaps
    



def get_train_test_samples2(Data_PRE, Data_POST, Data_FUTURE, sea_col_Cap, prdMaps , DOW= False , train_val_percentage = 0.9  ,  FC_time_series = True , traffic_time_series = False ,  use_channels = False , seasenality_one_dimension = True ,  window = 10 ):
    
    
    
    if DOW:
        PRE_FC , PRE_Seas , PRE_Traf, PRE_TF_timeseries  =       dow_get_tensors2(Data_PRE , sea_col_Cap, prdMaps  , FC_time_series = FC_time_series , traffic_time_series = traffic_time_series ,  use_channels = use_channels , seasenality_one_dimension = seasenality_one_dimension ,   window = window)
        POST_FC , POST_Seas , POST_Traf , POST_TF_timeseries =  dow_get_tensors2(Data_POST , sea_col_Cap, prdMaps  , FC_time_series = FC_time_series , traffic_time_series = traffic_time_series ,  use_channels = use_channels , seasenality_one_dimension = seasenality_one_dimension ,   window = window)
        # FUTURE_FC , FUTURE_Seas , FUTURE_Traf ,FUTUR_TF_timeseries = dow_get_tensors2(Data_FUTURE , sea_col_Cap, prdMaps  , FC_time_series = False , traffic_time_series = True ,  use_channels = True , seasenality_one_dimension = True ,   window = window)


    else:
        PRE_FC , PRE_Seas , PRE_Traf , PRE_TF_timeseries =      get_tensors2(Data_PRE, sea_col_Cap, prdMaps  , FC_time_series = FC_time_series , traffic_time_series = traffic_time_series ,  use_channels = use_channels , seasenality_one_dimension = seasenality_one_dimension ,   window = window)
        POST_FC , POST_Seas , POST_Traf , POST_TF_timeseries = get_tensors2(Data_POST, sea_col_Cap, prdMaps , FC_time_series = FC_time_series , traffic_time_series = traffic_time_series ,  use_channels = use_channels , seasenality_one_dimension = seasenality_one_dimension ,   window = window)
        # FUTURE_FC , FUTURE_Seas , FUTURE_Traf , FUTURE_TF_timeseries = get_tensors2(Data_FUTURE, sea_col_Cap, prdMaps , FC_time_series = False , traffic_time_series = True ,  use_channels = True , seasenality_one_dimension = True ,   window = window)

    # Train/Val Spilit:
    # TODO: THIS SHOULD BE CHANGED TO RANDOMIZED.
    train_val_cutoff = round(PRE_FC.shape[0]*train_val_percentage) 


    # prepare train/val/test datasets
    PRE_FC_train = PRE_FC[:train_val_cutoff, :]
    PRE_FC_val = PRE_FC[train_val_cutoff:, :]

    PRE_Seas_train = PRE_Seas[:train_val_cutoff, :]
    PRE_Seas_val = PRE_Seas[train_val_cutoff:, :]

    PRE_Traf_train = PRE_Traf[:train_val_cutoff, :]
    PRE_Traf_val = PRE_Traf[train_val_cutoff:, :]

    PRE_TF_timeseries_train = PRE_TF_timeseries[:train_val_cutoff, :]
    PRE_TF_timeseries_val = PRE_TF_timeseries[train_val_cutoff:, :]



    train = [PRE_FC_train,PRE_Seas_train,PRE_TF_timeseries_train,PRE_Traf_train]
    val = [PRE_FC_val,PRE_Seas_val,PRE_TF_timeseries_val,PRE_Traf_val]
    test = [POST_FC , POST_Seas ,  POST_TF_timeseries , POST_Traf]
    
    return train, val, test
    

## Models:

In [10]:
# Kronos Model
import os
from keras import backend as K 
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Reshape , BatchNormalization, LSTM, Concatenate, Dense, Activation, Flatten, Conv2D, Conv1D, ConvLSTM2D, Conv3D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, History, ModelCheckpoint
import random


def kronos_32s_model(para_epochs, para_early_stop, para_model_name, para_sea_len, para_sea_dense, window, train_list, val_list, test_list):
        
    # Set Random Seed 
    seed_value = 44
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)

    
    # extract train/val/test datasets  ** since TF.keras only accepts channels_last we have to move axix **
    train_fc, train_sea, train_traf_time, train_output = train_list
    val_fc, val_sea, val_traf_time, val_output = val_list
    test_fc, test_sea, test_traf_time, test_gold_output = test_list
    
    # Reshape for Conv3d with channel 1.
    train_fc, val_fc, test_fc = train_fc.reshape(list(train_fc.shape)+[1]), val_fc.reshape(list(val_fc.shape)+[1]), test_fc.reshape(list(test_fc.shape)+[1]) 
    
    
    # Model Parameters:
    batch_size = 100
    early_stop = 100
    filter_num = 8
    rec_act = 'relu'
    rec_dropout = 0.1
    dropout = 0.1
    lstm_size = 64
    patience= early_stop
    batch_size_nbr = batch_size
    Error_Method = 'MC_dropout'
    B = batch_size
    # 128, 64,64

    # Input sizes: 
    K.clear_session()
    input_tensor_closure = Input(shape = (2,7,10,1)) # lets use 3D with Channel of 1
    input_tensor_seas = Input(shape = (window ,para_sea_len))
    input_tensor_seas = Input(shape = (para_sea_len))
    input_tensor_trrafic_tseries = Input(shape = (window ,2,7,10))

    # Feed the TF-t-series to ConvLSTM:
    cnn_lstm_tf_time = ConvLSTM2D(filters = filter_num , kernel_size = 3, padding = 'same', data_format = 'channels_first', 
                                  recurrent_activation = rec_act, recurrent_dropout = rec_dropout, dropout=dropout ,  return_sequences = False)(input_tensor_trrafic_tseries)
    cnn_lstm_tf_time = BatchNormalization(axis = 1)(cnn_lstm_tf_time)
    # cnn_lstm_tf_time = ConvLSTM2D(filters = filter_num//2 , kernel_size = 2, padding = 'same', data_format = 'channels_first', 
    #                               recurrent_activation = rec_act, recurrent_dropout = rec_dropout, dropout=dropout ,  return_sequences = False)(cnn_lstm_tf_time)
    # cnn_lstm_tf_time = BatchNormalization(axis = 1)(cnn_lstm_tf_time)
    cnn_lstm_tf_time = Flatten()(cnn_lstm_tf_time)
    cnn_lstm_tf_time = Dense(32, activation=rec_act)(cnn_lstm_tf_time)

    # print(cnn_lstm_tf_time)


    # # layer for seasonality Using a RNN (Here GRU instead of LSTM)
    # rnn_seas = GRU(lstm_size, recurrent_dropout = rec_dropout, dropout=dropout  , return_sequences = False  )(input_tensor_seas)
    # # rnn_seas = Bidirectional(GRU(lstm_size//2, recurrent_dropout = rec_dropout, dropout=dropout  , return_sequences = False ))(rnn_seas)
    # rnn_seas = BatchNormalization()(rnn_seas)
    # rnn_seas = Dense(64, activation=rec_act)(rnn_seas)

    # Seas No RNN:
    rnn_seas = Dense(128, activation=rec_act)(input_tensor_seas)
    rnn_seas = Dense(32, activation=rec_act)(rnn_seas)


    # print(rnn_seas)


    # layer for FC:
    cnn_FC = Conv3D(filters = filter_num , kernel_size = 3, strides = (1, 1, 1), padding = 'same', data_format = 'channels_last')(input_tensor_closure)
    cnn_FC = Conv3D(filters = filter_num//2 , kernel_size = 3, strides = (1, 1, 1), padding = 'same', data_format = 'channels_last')(cnn_FC)
    cnn_FC = BatchNormalization()(cnn_FC)
    cnn_FC = Flatten()(cnn_FC)
    cnn_FC = Dense(32, activation=rec_act)(cnn_FC)

    # print(cnn_FC)


    # concat frac_closure and seasonality and TF_sries
    FF_concat_all = Concatenate()([cnn_lstm_tf_time, rnn_seas, cnn_FC])
    # FF_concat_all = Dense(256, activation=rec_act)(FF_concat_all)
    FF_concat_all = Dense(140, activation=rec_act)(FF_concat_all)

    # # Output format:
    output_tensor = Reshape((2, 7, 10))(FF_concat_all)

    # print(output_tensor)


    # Kronos model setting
    Kronos_model = Model(inputs = [ input_tensor_closure, input_tensor_seas, input_tensor_trrafic_tseries], outputs = output_tensor)
    Kronos_model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    
    if para_early_stop: # with early stop
        
        os.chdir(r"\\corpaa.aa.com\campusshared\HDQ\HDQ_REVMGMT_Share\RMDEPT")
        path = "BFox/Kronos/Prototype/Output/FullPeriod/"
        market_nm = orig + dest
        # if not os.path.exists(path):
            # os.mkdir(path)
        
        # history = History()
        metric = 'val_accuracy'
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=patience , restore_best_weights=True)
        mc = ModelCheckpoint(path + '//' + market_nm + '_' + 'best_model.h5', monitor='val_loss', mode='auto', verbose=0, save_best_only=True, save_freq='epoch')
        base_hist = Kronos_model.fit([train_fc, train_sea, train_traf_time], train_output, epochs = para_epochs, 
                                 batch_size = batch_size_nbr, validation_data = ([ val_fc, val_sea, val_traf_time], val_output), verbose = 0, callbacks=[es,mc])
    else: # no early stop
        base_hist = Kronos_model.fit([train_fc, train_sea, train_traf_time], train_output, epochs = para_epochs, 
                                 batch_size = batch_size_nbr, validation_data = ([ val_fc, val_sea, val_traf_time], val_output) , verbose = 0)
         
    
    # use the trained model to predict
    test_pred = Kronos_model.predict([ test_fc, test_sea, test_traf_time])

    return test_pred, base_hist, Kronos_model


def test_acc(prediction_results, gold_labels):
    results = pd.DataFrame()
    test_size = gold_labels.shape[0]

    gold_tr_reshaped = gold_labels.reshape(test_size*14,10)
    pred_tr_reshaped = kronos32s_test_results3.reshape(test_size*14,10)

    # Gold, top, mid, bot
    results['gold_top_tr'] = gold_tr_reshaped[:,:3].sum(1)
    results['gold_mid_tr'] = gold_tr_reshaped[:,3:7].sum(1)
    results['gold_bot_tr'] = gold_tr_reshaped[:,7:].sum(1)
    results['gold_sum_tr'] = gold_tr_reshaped.sum(1)

    # Pred Top, Mid, Bot
    results['pred_top_tr'] = pred_tr_reshaped[:,:3].sum(1)
    results['pred_mid_tr'] = pred_tr_reshaped[:,3:7].sum(1)
    results['pred_bot_tr'] = pred_tr_reshaped[:,7:].sum(1)
    results['pred_sum_tr'] = pred_tr_reshaped.sum(1)

    # all FvT errors
    results['top_FvT'] = results['pred_top_tr'] - results['gold_top_tr']
    results['mid_FvT'] = results['pred_mid_tr'] - results['gold_mid_tr']
    results['bot_FvT'] = results['pred_bot_tr'] - results['gold_bot_tr']
    results['sum_FvT'] = results['pred_sum_tr'] - results['gold_sum_tr']

    # squared FvT errors for MSE
    results['top_FvT_sqr'] = results['top_FvT']**2
    results['mid_FvT_sqr'] = results['mid_FvT']**2
    results['bot_FvT_sqr'] = results['bot_FvT']**2
    results['sum_FvT_sqr'] = results['sum_FvT']**2
    
    result_sum = { 
                    "top_FvT" : [results['top_FvT'].mean(), results['top_FvT'].std(), results['top_FvT_sqr'].mean()],
                    "mid_FvT" : [results['mid_FvT'].mean(), results['mid_FvT'].std(), results['mid_FvT_sqr'].mean()],
                    "bot_FvT" : [results['bot_FvT'].mean(), results['bot_FvT'].std(), results['bot_FvT_sqr'].mean()],
                    "sum_FvT" : [results['sum_FvT'].mean(), results['sum_FvT'].std(), results['sum_FvT_sqr'].mean()]
                }
    return results , result_sum



## Training Loop:

In [ ]:
import pickle


def store_data(data, path, file_name):
    save_path = os.path.join(path, f"{file_name}.pickle" )
    with open(save_path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
import warnings
warnings.filterwarnings('ignore')

print("Ignoring the warnings")

Ignoring the warnings


In [ ]:
from tqdm.notebook import trange, tqdm
from collections import defaultdict
from sklearn.utils import shuffle

switch_orig_dist = False
orig = "PHX"

yesterday =  datetime.today() - timedelta(days=2)
next_year_today = datetime.today() + timedelta(days=365)

pull_start = '2017-09-01'
pull_end = next_year_today.strftime("%Y-%m-%d")

# Pre: pre-covid period, used for train and validation
Pre_start, Pre_end = '2017-09-01', '2020-01-30'
# Post: post-covid period, used for test
Post_start, Post_end = '2021-07-01',  yesterday.strftime("%Y-%m-%d")
# Future: Today till one year in future:
future_start , future_end = datetime.today().strftime("%Y-%m-%d") ,   next_year_today.strftime("%Y-%m-%d")

new_market= False
ulcc_list = ['NK','SY','F9'] # Spirit SunCountry Frontier 

# Extracting for Seas:
sea_col_fcst = ['week_x', 'week_y', 'forecastDayOfWeek','avgrasm','dowavgrasm', 'forecastId'] #+ forecastDayOfWeek, FCST
sea_col_Cap = ['week_x', 'week_y','dow_x', 'dow_y', 'avgrasm','seats_AA_fcst','seats_OA_fcst','seats_ulcc_fcst' , 'seats_AA' , 'seats_OA' , 'seats_ulcc']
sea_col = ['week_x', 'week_y', 'dow_x', 'dow_y','avgrasm','dowavgrasm']

# Data reshaping parameters:
train_val_percentage = .9
time_series = False
seasenality_one_dimension = False 
window = 0 

# Model parameters;
epochs = 150
early_stop = 10
sea_dense = 128




kronos_3_timeseries = defaultdict()

hcrt, mos, az = connect_to_servers()


# Pull all the dest given rig.
all_dest = find_all_dest_given_leg(orig , hcrt)
# all_dest.pop(0) # Remove Austin

print(f"There are {len(all_dest)} FCSTs from {orig}")

if switch_orig_dist:
    main_dest = orig

for dest in tqdm(all_dest):
    if switch_orig_dist:
        dest , orig = main_dest, dest
    print(f" Flights from {orig} to {dest}:")
    # Pull all the FCSTs
    fcst_id_df = get_fcst_given_leg(orig, dest, hcrt )   
    
    if len(fcst_id_df) == 0:
        print(f" *** Fot the {orig}-{dest}, there is no avilable FCSTs! *** ")
        continue 
    # Pull OAG:
    oag_df = get_oag_data(orig,dest, pull_start, pull_end)
    
    #Pull prdMaps:
    prdMaps = get_prdMaps(orig, dest, hcrt)
    

    # Processing: OAG Per Day:
    oag_kl_total_Per_Day_and_AA = oag_per_day(oag_df)
    
    all_tensors = [[] for i in range(12)]
    
    for _,_ , fcst_id , fcst_start , fcst_end in tqdm(fcst_id_df.values):
        
        hcrt, mos, az = connect_to_servers()
        
        print(f" ------------- ****** {orig}-{dest}-{fcst_id}) ****** ------------- ")
        
        # print( fcst_id , fas, adf )
        print( f"fcst_start and fcst_end for {orig}-{dest} at FCST_ID {fcst_id} are: {fcst_start}, {fcst_end}") 
        
        #  Processing: OAG per FCST:
        oag_kl =  oag_per_fcst(oag_df, fcst_start, fcst_end )
        
        # Merge and Normalize: OAG per FCST and OAG per Day:
        oag_kl_fcst_total = pd.merge(oag_kl,oag_kl_total_Per_Day_and_AA ,on = "adj_dep_date", how='left',suffixes=('_fcst', '_day'))
        oag_kl_fcst_total = normalize_oag_kl_fcst_total(oag_kl_fcst_total)
        
        
        # Pull data from the file: pullData_FullPeriod.py
        df = pull_data(orig,dest,fcst_id,new_market)
        df = pull_seas(df, orig, dest)
        
        if len(df) < 100:
            print(f"insufficent data for market ({orig}-{dest}-{fcst_id}), IGNORED")
            continue
            
        
        df['flightDepartureDate'] = pd.to_datetime(df['flightDepartureDate'], format='%Y/%m/%d')

        # Merge new features (including the total day seats) into current Kronos dataset by dep_date
        df = pd.merge(df,oag_kl_fcst_total, left_on=['flightDepartureDate'],\
              right_on=['adj_dep_date'], how='left')
        df.dropna(inplace=True)
        
        # processing: Group and pad the DF:
        post = group_and_pad(df)
        
        # Cut the 'post' data into Pre-covid and Post-covid parts
        Data_PRE = post[ (post['flightDepartureDate']>=Pre_start) & (post['flightDepartureDate']<=Pre_end) ]
        Data_POST = post[ (post['flightDepartureDate']>=Post_start) & (post['flightDepartureDate']<=Post_end) ]
        Data_FUTURE = post[ (post['flightDepartureDate']>=future_start) & (post['flightDepartureDate']<=future_end) ]
        
        print(f" For market ({orig}-{dest}-{fcst_id}) , we have {Data_PRE.shape[0]/14} Pre-Covid data, {Data_POST.shape[0]/14} Post-Covid data, and {Data_FUTURE.shape[0]/14} future data (from now to one year from now)")
        
        # TODO: This can be edited later, so we use the POST data to train....
        if len(Data_PRE) <= len(Data_POST):
            print(f" *** The Pre Covid data is less than the post covid data, so we ignore {orig}-{dest}-{fcst_id} market ***")
            continue
        
#         # TODO: When merging models together this can be useefull for training porposes, but for now not usefull.
        if len(Data_FUTURE)//14 <= 5 * 7:
            print(f" *** There is no future flights for the {orig}-{dest}-{fcst_id} market, so ignored! *** ")
            continue
        
        if any([Data_PRE.shape[0]/14 <= 10 * 7 , Data_POST.shape[0]/14 <= 10 * 7]):
            print(f" *** Low amount of data for either PRE, or POST of {orig}-{dest}-{fcst_id} market, so ignored! *** ")
            continue
                
        # MErge FCSTs:
            
        # Train Kronos 2 (No Additional Feat (using sea_col)):
        use_channels = True
        seasenality_one_dimension = True
        DOW= True
        FC_time_series = False
        traffic_time_series = True 
        window = 10
        train, val, test = get_train_test_samples2(Data_PRE, Data_POST, Data_FUTURE, sea_col_fcst, prdMaps , DOW= DOW , train_val_percentage = train_val_percentage ,  FC_time_series = FC_time_series , traffic_time_series = traffic_time_series,  use_channels = use_channels , seasenality_one_dimension = seasenality_one_dimension ,  window = window )
        
        [all_tensors[i].append(item) for i,item in enumerate( train + val +test)]
        
    if len(all_tensors[0]) == 0:
        print(f"No data for {orig}-{dest} -> No Model!")
        continue
    # Now concatinate and shuffle the data:
    train = [np.concatenate(all_tensors[i]) for i in range(0,4)] 
    val = [np.concatenate(all_tensors[i]) for i in range(4,8)] 
    test = [np.concatenate(all_tensors[i]) for i in range(8,12)] 
    
    # Shuffle:
    train = shuffle(train[0],train[1],train[2],train[3])
    val = shuffle(val[0],val[1],val[2],val[3])
    
    
    kronos32s_test_results3 , kronos32s_hist3,  kronos32s_model3 = kronos_32s_model(para_epochs = 500, para_early_stop =  True, para_model_name = 'kronos32s', para_sea_len = len(sea_col_fcst), para_sea_dense =  160, window = window ,
             train_list = train , val_list = val, test_list = test)

    _ , results_summary = test_acc(kronos32s_test_results3, test[3])

    kronos_3_timeseries[f"{orig}-{dest}"] = results_summary

    print(f"val plot {orig}-{dest}")
    plt.plot(kronos32s_hist3.history['loss'])
    plt.plot(kronos32s_hist3.history['val_loss'])
    plt.title('loss')
    plt.ylabel('MSE')
    plt.xlabel('epoch')
    plt.legend(['train','val'] , loc = "upper left")
    plt.show()
        

        

store_data(kronos_3_timeseries, ".", "kronos3_results")        

There are 11 FCSTs from LAS


  0%|          | 0/11 [00:00<?, ?it/s]

 Flights from LAS to CLT:


  0%|          | 0/7 [00:00<?, ?it/s]

 ------------- ****** LAS-CLT-1) ****** ------------- 
fcst_start and fcst_end for LAS-CLT at FCST_ID 1 are: 180, 431
 For market (LAS-CLT-1) , we have 816.0 Pre-Covid data, 474.0 Post-Covid data, and 210.0 future data (from now to one year from now)
 ------------- ****** LAS-CLT-2) ****** ------------- 
fcst_start and fcst_end for LAS-CLT at FCST_ID 2 are: 432, 548
 For market (LAS-CLT-2) , we have 577.0 Pre-Covid data, 77.0 Post-Covid data, and 0.0 future data (from now to one year from now)
 *** There is no future flights for the LAS-CLT-2 market, so ignored! *** 
 ------------- ****** LAS-CLT-3) ****** ------------- 
fcst_start and fcst_end for LAS-CLT at FCST_ID 3 are: 549, 637
 For market (LAS-CLT-3) , we have 348.0 Pre-Covid data, 454.0 Post-Covid data, and 196.0 future data (from now to one year from now)
 ------------- ****** LAS-CLT-4) ****** ------------- 
fcst_start and fcst_end for LAS-CLT at FCST_ID 4 are: 638, 798
 For market (LAS-CLT-4) , we have 284.0 Pre-Covid data, 4